# Crawling Berita Kompas

# 1. Library Yang Dibutuhkan

In [ ]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd

# 2. Crawling Berita Kompas

Crawling data atau web crawling adalah proses ekstraksi informasi dari situs web dengan menggunakan program komputer. Tujuan utama dari web crawling adalah untuk mengumpulkan data dari berbagai sumber di internet.
Terdapat berbagai macam library yang dapat digunakan untuk crawling data. Library yang digunakan untuk crawling berita Kompas yaitu Beautiful Soup dan Requests. Beautiful Soup Digunakan untuk mengekstrak informasi dari HTML dan Request digunakan untuk mengirim permintaan HTTP dan mengakses halaman web.

In [ ]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd

csv = {"Judul": [], "Berita": [], "Topik": []}
max_berita_per_topik = 1  # Tingkatkan batas menjadi 200

# Daftar topik khusus yang ingin Anda ambil
target_topics = ["BOLA", "EDUKASI", "OTOMOTIF", "NEWS"]

# Kamus untuk menyimpan hitungan artikel yang diambil untuk setiap topik
berita_count = {topic: 0 for topic in target_topics}
berita_count["SPORT"] = 0  # Inisialisasi hitungan untuk SPORT

for i in range(1, 10):
    url = "https://indeks.kompas.com/?page={}".format(i)
    client = requests.get(url)
    page_html = client.content
    page_soup = soup(page_html, "html.parser")
    berita = page_soup.findAll("div", {"class": "latest--indeks mt2 clearfix"})

    for h in berita:
        r = requests.get(h.select_one('a.article__link')['href'])
        page = soup(r.content, "html.parser")
        halaman_isi = page.select_one("div", {"class": "col-bs10-10"})

        judul_berita = halaman_isi.select_one("h1.read__title")
        judul = judul_berita.text if judul_berita else ""

        isi_berita = halaman_isi.findAll('p')
        isi = '\n'.join([p.get_text() for p in isi_berita[3:-1] if "Baca juga" not in p.get_text()])

        topik_berita = h.find("div", {"class": "article__list__info"}).select("div.article__subtitle.article__subtitle--inline")
        topik = topik_berita[0].text if topik_berita else ""

        # Memeriksa apakah topik berita saat ini termasuk dalam topik yang diinginkan
        if topik.upper() in target_topics:
            # Memeriksa jumlah berita untuk topik tertentu
            jumlah_berita_topik_ini = berita_count[topik]
            if jumlah_berita_topik_ini < max_berita_per_topik:
                # Memperbarui topik jika itu "BOLA"
                if topik.upper() == "BOLA":
                    topik = "SPORT"
                csv["Judul"].append(judul)
                csv["Berita"].append(isi)
                csv["Topik"].append(topik)
                berita_count[topik] += 1  # Menambahkan hitungan untuk topik saat ini

    # Menyimpan data yang ada pada kamus ke dalam file CSV setelah selesai iterasi halaman
    data = pd.DataFrame(csv)
    data.to_csv("data_berita.csv", index=False)


TypeError: ignored